In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)


In [ ]:
import pandas as pd

import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')
import numpy as np
import random
from collections import OrderedDict
import matplotlib.pyplot as plt
import openeye.oechem as oechem
import openeye.oeiupac as oeiupac
import openeye.oeomega as oeomega
import openeye.oeshape  as oeshape
import openeye.oedepict as oedepict
import openeye.oegrapheme as oegrapheme
#[name for _, name, _ in pkgutil.iter_modules(['openeye'])]

In [ ]:
!pip install py3Dmol

In [ ]:
omega = oeomega.OEOmega() #Initialize class
omega.SetMaxConfs(5) #Here we want to use more conformers if needed
omega.SetStrictStereo(False) #Set to false to pick random stereoisomer if stereochemistry is not specified
omega.SetStrictAtomTypes(False) #Be a little loose about atom typing to ensure parameters are available to omega for all molecules
#In this case the 'StrictStereo' parameter above will also matter. Since tramadol has an unspecified stereocenter,
#this will cause Omega to pick a random stereoisomer. Without specifying this, Omega will refuse to do anything.

In [ ]:
fitmol = oechem.OEMol()
refmol = oechem.OEMol()


mol_name = 'random'
oechem.OESmilesToMol(fitmol, 'N[C@@H](CCC1)CN1C([C@H](c1ccccc1)NC(c(cc1)cc2c1ocn2)=O)=O')
# oechem.OESmilesToMol(refmol, '')


# omega(refmol)
# refmol.SetTitle('Refmol')
# ofs = oechem.oemolostream('ref.mol2')
# oechem.OEWriteMolecule(ofs, refmol)
# ofs.close()


omega(fitmol)
fitmol.SetTitle('Fitmol')


In [ ]:
outfs = oechem.oemolostream('fitted_%s_output.mol2'% mol_name)


istream = oechem.oemolistream('ref_Emin.mol2')
for i in istream.GetOEGraphMols():
  refmol = oechem.OEMol(i)

options = oeshape.OEROCSOptions()
options.SetNumBestHits(5)
options.SetConfsPerHit(20)
rocs = oeshape.OEROCS(options)
rocs.AddMolecule(fitmol) #Add our molecule as the one we are fitting

for res in rocs.Overlay(refmol):
    outmol = res.GetOverlayConfs() #Use GetOverlayConf to get just the best; GetOverlayConfs for all
    # oeshape.OERemoveColorAtoms(outmol)
    # oechem.OEAddExplicitHydrogens(outmol)
    oechem.OEWriteMolecule(outfs, outmol)
    print("title: %s  tanimoto combo = %.2f" % (outmol.GetTitle(), res.GetTanimotoCombo()))
outfs.close()


In [ ]:
import py3Dmol
#First we assign the py3Dmol.view as viewer
viewer = py3Dmol.view()
viewer.addModel(open('ref.mol2', 'r').read(),'mol2')
viewer.addModel(open('fitted_%s_output.mol2' % mol_name, 'r').read(),'mol2')
viewer.setBackgroundColor('0Xffffffff')
viewer.setStyle({'stick':{}})
#Zooming into all visualized structures
viewer.zoomTo()
#And we finally visualize the structures using the command below
viewer.show()
#viewer.pngURI()


In [ ]:

viewer2 = py3Dmol.view()
# viewer.addModel(open('PF-846_binding.mol2', 'r').read(),'mol2')

viewer2.addModel(open('fitted_%s_output.mol2' % mol_name, 'r').read(),'mol2')
viewer2.setBackgroundColor('0Xffffffff')
viewer2.setStyle({'stick':{}})
#Zooming into all visualized structures
viewer2.zoomTo()
#And we finally visualize the structures using the command below
viewer2.show()
# viewer2.pngURI()



viewer3 = py3Dmol.view()
viewer3.addModel(open('ref.mol2', 'r').read(),'mol2')
viewer3.setBackgroundColor('0Xffffffff')
viewer3.setStyle({'stick':{}})
#Zooming into all visualized structures
viewer3.zoomTo()
#And we finally visualize the structures using the command below
viewer3.show()
# viewer2.pngURI()



rdkit

In [ ]:
!pip install rdkit

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdDepictor
rdDepictor.SetPreferCoordGen(True)
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import SVG, Image
IPythonConsole.molSize = (600,400)

In [ ]:
que_smiles = "O=C([C@@H](C(C1)Cc2c1cccc2)NC(c(cc1)cc2c1ocn2)=O)NC[C@@H]1CNCCC1"
que_mol = Chem.MolFromSmiles(que_smiles)
rdDepictor.Compute2DCoords(que_mol)
que_mol

In [ ]:
ref_smiles = "O=C(C1=CC=C(N2C(N=CC=C3)=C3N=N2)C=C1)N([C@H]4CNCCC4)C5=C(Cl)C=CC=N5"
ref_mol = Chem.MolFromSmiles(ref_smiles)
rdDepictor.Compute2DCoords(ref_mol)
ref_mol

In [ ]:
que_mc = Chem.Mol(que_mol.ToBinary())  # make a copy of the Mol object
que_mol.GetSubstructMatch(ref_mol)
que_mol

openeye

In [ ]:
cff = oeshape.OEColorForceField()
cff.Init(oeshape.OEColorFFType_ImplicitMillsDean)
qdisp = oegrapheme.OEShapeQueryDisplay(refmol, cff)

opts = oegrapheme.OEShapeOverlapDisplayOptions()
opts.SetTitleLocation(oedepict.OETitleLocation_Hidden)
arcpen = oedepict.OEPen(oechem.OEGrey, oechem.OEGrey, oedepict.OEFill_Off,
                        2.0, oedepict.OEStipple_ShortDash)
opts.SetQuerySurfaceArcFxn(oegrapheme.OEDefaultArcFxn(arcpen))
odisp = oegrapheme.OEShapeOverlapDisplay(qdisp, fitmol, opts)

image = oedepict.OEImage(420.0, 280.0)
oegrapheme.OERenderShapeOverlap(image, odisp)
oegrapheme.OEDraw2DSurface(image)


In [ ]:
!pip install mdtraj

In [ ]:
!pip install nglview

# reproduce example

In [ ]:
#Let's first generate morphine and tramadol from their names
morphine= OEMol()
OEParseIUPACName(morphine, 'morphine')
#Make sure we get 'True' so we know it worked:

In [ ]:
#Let's first generate morphine and tramadol from their names
morphine= OEMol()
OEParseIUPACName(morphine, 'morphine')
#Make sure we get 'True' so we know it worked:

In [ ]:
#In older toolkit versions that doesn't work; if not, pull the IUPAC name from Wikipedia.
OEParseIUPACName(tramadol, '2-[(Dimethylamino)methyl]-1-(3-methoxyphenyl)cyclohexanol')

In [ ]:
omega = OEOmega() #Initialize class
omega.SetMaxConfs(100) #Here we want to use more conformers if needed
omega.SetStrictStereo(False) #Set to false to pick random stereoisomer if stereochemistry is not specified
omega.SetStrictAtomTypes(False)

In [ ]:
refmol = morphine
omega(refmol)

#Now we generate 3D conformations for tramadol and label it our 'fitmol', the molecule to be fit onto the reference
fitmol = tramadol
omega(fitmol)
fitmol.SetTitle('tramadol')

#And just so we can see how this works, let's write out morphine to a mol2 file for viewing later
ofs = oemolostream('ref.mol2')
OEWriteMolecule(ofs, refmol)
ofs.close()

In [ ]:
# Open output stream for output molecule
outfs = oemolostream('fitted_output.mol2')
# How many conformers will we generate for fitting?
nconfs = 20

In [ ]:
# Setup ROCS to provide specified number of conformers per hit
options = OEROCSOptions()
options.SetNumBestHits(1)
options.SetConfsPerHit(nconfs)
rocs = OEROCS(options)
rocs.AddMolecule(fitmol) #Add our molecule as the one we are fitting

In [ ]:
# Loop over results and output
for res in rocs.Overlay(refmol):
    outmol = res.GetOverlayConf() #Use GetOverlayConf to get just the best; GetOverlayConfs for all
    OERemoveColorAtoms(outmol)
    OEAddExplicitHydrogens(outmol)
    OEWriteMolecule(outfs, outmol)
    print("title: %s  tanimoto combo = %.2f" % (outmol.GetTitle(), res.GetTanimotoCombo()))
outfs.close()

In [ ]:
import py3Dmol
#First we assign the py3Dmol.view as viewer
viewer = py3Dmol.view()
viewer.addModel(open('ref.mol2', 'r').read(),'mol2')
viewer.addModel(open('fitted_output.mol2', 'r').read(),'mol2')
viewer.setStyle({'stick':{}})
#Zooming into all visualized structures
viewer.zoomTo()
#And we finally visualize the structures using the command below
viewer.show()

In [ ]:

from google.colab import output
output.enable_custom_widget_manager()
import mdtraj, nglview
ref = mdtraj.load('ref.mol2')
view = nglview.NGLWidget()
view.add_trajectory(ref)
##view.clear_representations()
#view.add_representation("ball+stick")
# fitted = mdtraj.load('fitted_%s_output.mol2'%mol_name)
# view.add_trajectory(fitted)
 #Use this and the below to switch one to blue sticks to make it easier to see
#view[0].add_ball_and_stick(color='blue')
view